In [1]:
#Load
#Load Libraries
import pandas as pd
import numpy as np
import random
from sqlalchemy import create_engine
import pickle

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
#Create CSR_Matrix
from scipy.sparse import csr_matrix
train=train.drop_duplicates(
['idx','game_name'])
users_combined_pivot = train.pivot(index = 'game_name',
                                           columns = 'idx',
                                           values = {"sentiment_R", "like_degree_2w","like_degree_f","price_usd",
                                                    "is_free","released_year","Action","Adventure",
                                                    "Animation&Modeling","Casual","Design &  illustration", "Documentary","Episodic",
                                                     "Early Access","Gore", "Indie", "Massively Multiplayer", "Movie", "Nudity",
                                                     "Photo Editing", "Racing", "RPG", "Sexual Content", "Simulation", "Sports",
                                                     "Strategy", "Utilities", "Video Production", "Webpublishing"}).fillna(0)


In [4]:
# load the model from disk
filename = "finalized_model.sav"
knn_model = pickle.load(open(filename, 'rb'))


In [8]:
#Randomly Get a User
test_user = list(test['idx'])
test_user =list(set(test_user))
target_user = np.random.choice(test_user)
target_gamelist = []
user_game_t = list(zip(train.idx,train.game_name))
for items in user_game_t:
    if items[0] == target_user:
        target_gamelist.append(items[1])

#Get the User's Current PlayList With its Log_Playtime_2Weeks
num_games = users_combined_pivot.shape[0]
temp = list(zip(train.idx,zip(train.game_name,train.log_playtime_2weeks)))
result = []
target_gamelist_i = []
target_log = []
for items in target_gamelist:
    for i in range(num_games):
        if users_combined_pivot.index[i] == items:
            target_gamelist_i.append((i,items))
for items in target_gamelist:
    for i in temp:
        if i[1][0] == items and i[0] == target_user:
          target_log.append((items, i[1][1]))

In [9]:
#Get the Recommendations for Each Game in User's PlayList
for items in target_gamelist_i:
    distances, indices = knn_model.kneighbors(
    users_combined_pivot.iloc[items[0], : ].values.reshape(1, -1),
    n_neighbors = 6)
    print("\n\n")
    for i in range(len(distances.flatten())):
        if (i == 0):
            print("Recommendations for {}:\n".format(
                users_combined_pivot.index[items[0]]))
  
        else:
            print("{i}: {title}, with distance of {dist}".format(i=i,
                                                                 title=users_combined_pivot.index[indices.flatten()[i]],
                                                                 dist=distances.flatten()[i]))
            result.append((users_combined_pivot.index[items[0]], (users_combined_pivot.index[indices.flatten()[i]],1-distances.flatten()[i])))




Recommendations for Red Matter:

1: Reigns: Game of Thrones, with distance of 1.0
2: Reign Of Kings, with distance of 1.0
3: Regency Solitaire, with distance of 1.0
4: Reigns, with distance of 1.0
5: Relic Hunters Legend, with distance of 1.0



Recommendations for SCUM:

1: Toribash, with distance of 0.9105378375010608
2: Insurgency: Sandstorm, with distance of 0.9812343985455384
3: Hunt: Showdown, with distance of 0.986337507212913
4: Squad, with distance of 0.9897471640989022
5: SteamVR, with distance of 0.9928658679590648



Recommendations for SteamVR:

1: The Lab, with distance of 0.9691974513603336
2: Rec Room, with distance of 0.9742319292069277
3: Tacopocalypse, with distance of 0.9788756933705656
4: Creature Clicker - Capture, Train, Ascend!, with distance of 0.978960959184343
5: Valentino Rossi The Game, with distance of 0.9789685069745258



Recommendations for Vermillion:

1: Reigns: Game of Thrones, with distance of 1.0
2: Reign Of Kings, with distance of 1.0
3: Regenc

In [10]:
#Get Top N Recommendations
rank = []
for i in result:
    for j in target_log:
        if i[0] == j[0]:
            rank.append((i[0],(i[1][0], i[1][1]*j[1])))
rank.sort(reverse = True, key =lambda x: x[1][1])
#Return Top 5 Recommendations 
print(f"Recommendations for User {target_user}:\n")
final_result = []
for i in range(5):
    final_result.append(rank[i][1][0])
    print(f"{rank[i][1][0]} with value {rank[i][1][1]}")

Recommendations for User 12643:

Toribash with value 0.5970076671962709
The Lab with value 0.20411979579258346
Rec Room with value 0.17075773208835765
Tacopocalypse with value 0.13998481768185198
Creature Clicker - Capture, Train, Ascend! with value 0.1394197852002889
